In [87]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import pandas as pd
from PIL import Image
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr
from time import time

In [2]:
from inrnet import args as args_module
from inrnet import inn, experiments, optim, util, losses, models, jobs as job_mgmt
from inrnet.data import dataloader
from inrnet.experiments import depth
import inrnet.inn.functional as inrF
from inrnet.models.inrs import siren, rff
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

# interactive

### cmd line

In [ ]:
python train.py -c=seg_nn

bash train.sh seg_nn3 seg_nn3
bash train.sh seg_i3 seg_i3
bash train.sh seg_nn5 seg_nn5
bash train.sh seg_i5 seg_i5

In [ ]:
bash train.sh inet_nn2 inet_nn2
bash train.sh inet_nn5 inet_nn5
bash train.sh inet_i2 inet_i2
bash train.sh inet_i5 inet_i5

python infer.py -c=inet_val -t=inet_inn_train
bash infer.sh i1x_i2 inet_val inet_i2
bash infer.sh i2x_i2 in2x_val inet_i2
bash infer.sh i1x_i5 inet_val inet_i5
bash infer.sh i2x_i5 in2x_val inet_i5
bash infer.sh i1x_nn2 inet_val inet_nn2
bash infer.sh i2x_nn2 in2x_val inet_nn2
bash infer.sh i1x_nn5 inet_val inet_nn5
bash infer.sh i2x_nn5 in2x_val inet_nn5

### objectfolder

In [ ]:
python OF_render.py --modality vision \
      --object_file_path /data/vision/polina/scratch/clintonw/datasets/objectfolder/ObjectFolder1-100/1/ObjectFile.pth \
      --vision_test_file_path ~/code/ObjectFolder/demo/vision_1.npy \
      --vision_results_dir ~/code/ObjectFolder/results


In [265]:
x = torch.linspace(-1,1,16, dtype=torch.float, device='cuda')

In [ ]:
df['material'][0]

In [278]:
df = df[df['material'].isin(('Ceramic', 'Polycarbonate', 'Wood'))]

In [ ]:
for n in df.index:
    print(n)

In [ ]:
df = pd.read_csv(osp.expanduser('~/code/ObjectFolder/objects.csv'),
                 names=['id', 'name', 'volume', 'material', 'url'], index_col=0)

In [256]:
df = pd.read_csv(osp.expanduser('~/code/ObjectFolder/objects300.csv'), index_col=0)

In [255]:
df[:300].to_csv(osp.expanduser('~/code/ObjectFolder/objects300.csv'))

In [224]:
out = np.load(osp.expanduser('~/code/ObjectFolder/demo/vision_1.npy'))

In [ ]:
out

In [223]:
np.save(osp.expanduser('~/code/ObjectFolder/demo/vision_1.npy'), out[:1])

In [ ]:
bash train.sh gen_nn4 gen_nn4
bash train.sh gen_i4 gen_i4
python train.py -c=gen_nn4

In [ ]:
bash train.sh warp_inn warp_train
bash train.sh warp_nn warp_nn

### random fourier features

In [16]:
paths = util.glob2('/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/*/test_*.pt')
# for path in paths:
#     os.remove(path)

In [ ]:
path = '/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/fine_val_0.pt'
sd,seg = torch.load(path)
sinr = siren.Siren()
sinr.load_state_dict(sd)
sinr = siren.to_black_box([sinr])
img = sinr.cuda().produce_images(256,256)

In [161]:
path = '/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/6/val_4.pt'
sd = torch.load(path)
sinr = siren.Siren()
sinr.load_state_dict(sd)
sinr = siren.to_black_box([sinr])
img = sinr.cuda().produce_images(256,256)

In [143]:
path = '/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/5/test_0.pt'
sd = torch.load(path)
inr = rff.RFFNet()
inr.load_state_dict(sd)
inr = rff.to_black_box([inr])
img2 = inr.cuda().produce_images(256,256)

In [ ]:
import imgviz
labelviz_pred = imgviz.label2rgb(seg.max(0).indices)
plt.imshow(labelviz_pred); plt.axis('off')

In [ ]:
util.imshow(img)

In [ ]:
util.imshow(torch.cat(((img+1)/2, img2), dim=-1))

In [163]:
m = torchvision.models.resnet18(pretrained=False)

### audio

In [92]:
df = pd.read_csv(f'{DS_DIR}/environmental-sound-classification/esc50.csv')
classes = np.array(df[['target', 'category']]).tolist()
classes = set(['{} {}'.format(c[0], c[1]) for c in classes])
classes = np.array([c.split(' ') for c in classes])
classes = {k: v for k, v in classes}

In [ ]:
classes

In [95]:
from inrnet.data.audio import utils
ESC50 = utils.ESC50

In [ ]:
train_splits = [1,2,3,4]
test_split = 5

shared_params = {'csv_path': f'{DS_DIR}/environmental-sound-classification/esc50.csv',
                 'wav_dir': f'{DS_DIR}/environmental-sound-classification/audio',
                 'dest_dir': f'{DS_DIR}/environmental-sound-classification/16000',
                 'audio_rate': 16000,
                 'only_ESC10': True,
                 'pad': 0,
                 'normalize': True}

train_gen = ESC50(folds=train_splits,
                  randomize=True,
                  strongAugment=True,
                  random_crop=True,
                  inputLength=2,
                  mix=True,
                  **shared_params).batch_gen(16)

test_gen = ESC50(folds=[test_split],
                 randomize=False,
                 strongAugment=False,
                 random_crop=False,
                 inputLength=4,
                 mix=False,
                 **shared_params).batch_gen(16)

X, Y = next(train_gen)

### swin

In [156]:
sd = torch.load('/data/vision/polina/users/clintonw/code/diffcoord/temp/upernet_deit.pth')['state_dict']

In [158]:
sd['backbone.patch_embed.projection.weight'].shape

torch.Size([384, 3, 16, 16])

In [157]:
sd.keys()

odict_keys(['backbone.cls_token', 'backbone.pos_embed', 'backbone.patch_embed.projection.weight', 'backbone.patch_embed.projection.bias', 'backbone.layers.0.ln1.weight', 'backbone.layers.0.ln1.bias', 'backbone.layers.0.attn.attn.in_proj_weight', 'backbone.layers.0.attn.attn.in_proj_bias', 'backbone.layers.0.attn.attn.out_proj.weight', 'backbone.layers.0.attn.attn.out_proj.bias', 'backbone.layers.0.ln2.weight', 'backbone.layers.0.ln2.bias', 'backbone.layers.0.ffn.layers.0.0.weight', 'backbone.layers.0.ffn.layers.0.0.bias', 'backbone.layers.0.ffn.layers.1.weight', 'backbone.layers.0.ffn.layers.1.bias', 'backbone.layers.1.ln1.weight', 'backbone.layers.1.ln1.bias', 'backbone.layers.1.attn.attn.in_proj_weight', 'backbone.layers.1.attn.attn.in_proj_bias', 'backbone.layers.1.attn.attn.out_proj.weight', 'backbone.layers.1.attn.attn.out_proj.bias', 'backbone.layers.1.ln2.weight', 'backbone.layers.1.ln2.bias', 'backbone.layers.1.ffn.layers.0.0.weight', 'backbone.layers.1.ffn.layers.0.0.bias', 'b

### fashionMNIST

In [194]:
ds = torchvision.datasets.FashionMNIST(root=DS_DIR, train=True, transform=transforms.ToTensor())
len(ds)
#inr = siren.Siren(out_channels=1, C=64, first_omega_0=20, hidden_omega_0=20)
# xy = util.meshgrid_coords(28,28, c2f=False)
# r = xy.norm(dim=-1)
# torch.sin(theta)
# maxr_theta = min()
# theta = torch.atan2(xy[:,1], xy[:,0])
# theta.max()
# plt.imshow(ds[0][0][0], cmap='gray')

60000

### etc

In [215]:
sd = torch.load('/data/vision/polina/users/clintonw/code/diffcoord/temp/wgan.pth')['state_dict']

In [181]:
base = torchvision.models.efficientnet_b0(pretrained=False, device='cuda')
optim = torch.optim.Adam(base.parameters())
optim.zero_grad()

In [ ]:
CUDA_VISIBLE_DEVICES=3 python train.py -j=inet2 -c=inet

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [97]:
base = torch.load(osp.expanduser('~/code/diffcoord/temp/upernet_convnext.pth'))

In [203]:
img = plt.imread('/data/vision/polina/scratch/clintonw/datasets/cityscapes/gtCoarse/val/frankfurt/frankfurt_000001_080091_gtCoarse_labelIds.png')

In [ ]:
base = torchvision.models.efficientnet_b0(pretrained=True)
base

In [42]:
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"
ds = torchvision.datasets.CIFAR10(root=DS_DIR, transform=transforms.ToTensor())

In [78]:
torch.cuda.empty_cache()
coords = util.meshgrid_coords(64,64)
#spacing = torch.tensor((4/127,4/127), device="cuda")
values = torch.randn(coords.size(0),6, device=coords.device)

In [218]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/diffcoord/inrnet")
query_string = "interpolate("
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

/data/vision/polina/users/clintonw/code/diffcoord/inrnet/inn/qmc.py
/data/vision/polina/users/clintonw/code/diffcoord/inrnet/data/oasis.py
/data/vision/polina/users/clintonw/code/diffcoord/inrnet/models/ddpm.py
/data/vision/polina/users/clintonw/code/diffcoord/inrnet/experiments/segment.py
/data/vision/polina/users/clintonw/code/diffcoord/inrnet/experiments/generate.py


### sympy

In [151]:
from sympy import sin, cos, tan, exp, log, integrate
from sympy.abc import a,b,c,n,m,x,y
from sympy import Point, Polygon
from sympy.integrals.intpoly import polytope_integrate

In [149]:
from sympy.functions.special import polynomials, spherical_harmonics

In [173]:
function = (3*polynomials.legendre(2, x)-2*polynomials.legendre(1, x) - 1) * \
    (.6*polynomials.legendre(4, y)-polynomials.legendre(1, y)+1.1)

In [ ]:
function

In [162]:
polygon = Polygon(Point(0, 0), Point(0, 1), Point(1, 1), Point(1, 0))
# polys = [1, x, y, x*y, x**2*y, x*y**2]
# expr = x*y

In [174]:
polytope_integrate(polygon, expr=function)

nan

In [146]:
polytope_integrate(polygon, expr=expr)

1/4

In [147]:
polytope_integrate(polygon, polys, max_degree=3)

{1: 1, x: 1/2, y: 1/2, x*y: 1/4, x**2*y: 1/6, x*y**2: 1/6}

### inet12

In [3]:
from inrnet.models.inrs import siren
keys = siren.get_siren_keys()

In [16]:
from robustness.tools.imagenet_helpers import ImageNetHierarchy, common_superclass_wnid
in_path = '/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch'
in_hier = ImageNetHierarchy(in_path, in_path)
superclass_wnid = common_superclass_wnid('big_12')
class_ranges, label_map = in_hier.get_subclasses(superclass_wnid,
                                                 balanced=True)

In [10]:
subpaths = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/train.txt",
            "r").read().split('\n')
labels = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/labels.txt",
            "r").read().split('\n')
labels = [l[:l.find(',')] for l in labels[:-1]]
class_labels = [np.array(labels)[list(cr)].tolist() for cr in class_ranges]

In [ ]:
sub_to_super = {}
for supercls,subclasses in enumerate(class_ranges):
    for subcls in subclasses:
        sub_to_super[subcls] = supercls

# big12path = f"{DS_DIR}/inrnet/big_12.pkl"
# pickle.dump((label_map, class_ranges, sub_to_super), open(big12path, 'wb'))
# label_map, class_ranges, sub_to_super = pickle.load(open(big12path, 'rb'))

In [ ]:
label_to_super = {}
for supercls,subclasses in enumerate(class_labels):
    for subcls in subclasses:
        label_to_super[subcls] = supercls

In [ ]:
N = 800
subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

for p in subpaths_by_cls:
    np.random.shuffle(p)

split_paths_by_cls = {'train':[p[:N] for p in subpaths_by_cls],
                     'test':[p[N:] for p in subpaths_by_cls]}

# big12path = f"{DS_DIR}/inrnet/big_12_labels.pkl"
# pickle.dump((split_paths_by_cls, class_labels, label_to_super), open(big12path, 'wb'))
# split_paths_by_cls, class_labels, label_to_super = pickle.load(open(big12path, 'rb'))

In [19]:
subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

for p in subpaths_by_cls:
    np.random.shuffle(p)

In [26]:
N = 23400
subpaths_by_cls = [p[:N] for p in subpaths_by_cls]

In [27]:
big12path = f"{DS_DIR}/inrnet/big_12_extra.pkl"
pickle.dump(subpaths_by_cls, open(big12path, 'wb'))
# subpaths_by_cls = pickle.load(open(big12path, 'rb'))

In [ ]:
paths = util.glob2('/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/train_*.pt')
for p in paths:
    weights, seg = torch.load(p)
    break

# Tables

### Classify

In [285]:
RESULTS_DIR = osp.expanduser("~/code/diffcoord/results")

model_jobs = {
    '2-layer CNN':('i1x_nn2', 'i2x_nn2'),
    'INR-2 (ours)':('i1x_i2', 'i2x_i2'),
    '5-layer CNN':('i1x_nn5', 'i2x_nn5'),
    'INR-5 (ours)':('i1x_i5', 'i2x_i5'),
    'EfficientNet \cite{efficientnet}':('i1x_nn', 'i2x_nn'),
    'INR-ft (ours)':('i1x_inn', 'i2x_inn'),
    'INR-mlp (ours)':('i1x_mlp', 'i2x_mlp'),
}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')
    if model.endswith('ours)'):
        print(r'\hline')

2-layer CNN & 66.2\% & 62.5\% \\
INR-2 (ours) & 41.9\% & 39.1\% \\
\hline
5-layer CNN & 68.2\% & 62.1\% \\
INR-5 (ours) & 48.1\% & 45.4\% \\
\hline
EfficientNet \cite{efficientnet} & 66.4\% & 59.8\% \\
INR-ft (ours) & 48.1\% & 40.5\% \\
\hline
INR-mlp (ours) & 32.4\% & 25.2\% \\
\hline


In [ ]:
jobs = ["raw_m_rgae", "dit_m_rgae", "raw_m_caae", "dit_m_caae",
        "raw_m_ipg", "dit_m_ipg", "raw_m_star", "dms_1"]
df = tables.get_results_table()
subtable = df.loc[jobs]

def latex_format(value, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$'%n_decimals).format(value)        
    else:
        return (r'${0:.%df}$'%n_decimals).format(value)
def plus_minus_format(mean, std, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$\scriptsize$\bm{{\pm {1:.%df}}}$'%(n_decimals, n_decimals)).format(mean, std)        
    else:
        return (r'${0:.%df}$\scriptsize$\pm {1:.%df}$'%(n_decimals, n_decimals)).format(mean, std)

def print_row(name, row, bold=()):
    field1 = latex_format(row["FID_tuned"], bold=1 in bold)
    field2 = latex_format(row["F_{1/8}"], bold=2 in bold)
    field3 = latex_format(row["F_8"], bold=3 in bold)
    field4 = plus_minus_format(row["age mean error"], row["age error STD"], bold=4 in bold)
    print(" & ".join([name, field1, field2, field3, field4]) + r" \\")

### Segment

In [ ]:
model_jobs = {'EffNet-T':('c1x_nn', 'c2x_nn', 'chx_nn'),
    'INR-tuned (ours)':('c1x_inn', 'c2x_inn', 'chx_inn'),
    'INR-scratch (ours)':('c1x_scr', 'c2x_scr', 'chx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

### Generate

In [ ]:
model_jobs = {'EffNet-T':('f1x_nn', 'f2x_nn', 'fhx_nn'),
    'INR-tuned (ours)':('f1x_inn', 'f2x_inn', 'fhx_inn'),
    'INR-scratch (ours)':('f1x_scr', 'f2x_scr', 'fhx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

# Figs

## for framework

In [ ]:
xy = util.generate_quasirandom_sequence(d=2, n=256)
plt.rcParams["figure.figsize"] = (2,1)
fig,ax = plt.subplots()
ax.scatter(xy[:,0], xy[:,1], s=2, c='k')
ax.plot();

In [ ]:
kernel = model.features[0][0].weight[3,1].detach()
plt.rcParams["figure.figsize"] = (.5,.5)
fig,ax = plt.subplots()
print(kernel.min().item(), kernel.max().item())
ax.imshow(kernel, vmin=-2.9, vmax=2.9);
plt.axis('off');

In [100]:
from scipy.interpolate import RectBivariateSpline as Spline2D
K = [2.5,2.5]
h,w=3,3
bbox = (-K[0]/2, K[0]/2, -K[1]/2, K[1]/2)
x,y = (np.linspace(bbox[0]/h*(h-1), bbox[1]/h*(h-1), h),
       np.linspace(bbox[2]/w*(w-1), bbox[3]/w*(w-1), w))
bs = Spline2D(x,y, kernel, bbox=bbox, kx=2,ky=2, s=0)
tx,ty,c = [torch.tensor(z).float() for z in bs.tck]
c = c.reshape(h,w)

In [106]:
H = 50
xy = util.meshgrid_coords(H,H).cpu()
w_oi = []
X = xy[:,0].unsqueeze(1)
Y = xy[:,1].unsqueeze(1)
px = py = 2

values, kx = (tx<=X).min(dim=-1)
values, ky = (ty<=Y).min(dim=-1)
kx -= 1
ky -= 1
kx[values] = tx.size(-1)-px-2
ky[values] = ty.size(-1)-py-2

Ctrl = c.view(1, *c.shape[-2:])
for z in range(X.size(0)):
    D = Ctrl[:, kx[z]-px : kx[z]+1, ky[z]-py : ky[z]+1].clone()

    for r in range(1, px + 1):
        try:
            alphax = (X[z,0] - tx[kx[z]-px+1:kx[z]+1]) / (
                tx[2+kx[z]-r:2+kx[z]-r+px] - tx[kx[z]-px+1:kx[z]+1])
        except RuntimeError:
            print("input off the grid")
        for j in range(px, r - 1, -1):
            D[:,j] = (1-alphax[j-1]) * D[:,j-1] + alphax[j-1] * D[:,j].clone()

    for r in range(1, py + 1):
        alphay = (Y[z,0] - ty[ky[z]-py+1:ky[z]+1]) / (
            ty[2+ky[z]-r:2+ky[z]-r+py] - ty[ky[z]-py+1:ky[z]+1])
        for j in range(py, r-1, -1):
            D[:,px,j] = (1-alphay[j-1]) * D[:,px,j-1].clone() + alphay[j-1] * D[:,px,j].clone()

    w_oi.append(D[:,px,py])

w = torch.stack(w_oi).view(xy.size(0))

In [107]:
k = w.reshape(H,H)

In [ ]:
fig,ax = plt.subplots()
print(k.min().item(), k.max().item())
ax.imshow(k, vmin=-2.9, vmax=2.9);
plt.axis('off');

## analysis

In [ ]:
from inrnet.experiments import classify
classify.analyze_change_resolution_grid_vs_qmc()

In [ ]:
ax = sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.5,.8,10)+torch.randn(10)/8, label="qmc");
sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.1,.8,10)+torch.randn(10)/8, label="grid", ax=ax);
ax.set_xlabel("Number of sample points");
ax.set_ylabel("ImageNet top-1 error");

### Deviation over depth

### Resolution vs. output (grid & QMC)

In [201]:
base_logits, grid_logits, grid_masks, qmc_logits, qmc_masks = torch.load(osp.expanduser(
    '~/code/diffcoord/temp/change_resolution_grid_vs_qmc.pt'))

In [212]:
grid_dists = (torch.cat(grid_logits, dim=0) - base_logits.cpu()).norm(dim=-1)
qmc_dists = (torch.cat(qmc_logits, dim=0) - base_logits.cpu()).norm(dim=-1)

In [ ]:
torch.sort(base_logits[0]).indices

In [ ]:
torch.sort(qmc_logits[-1][0]).indices

In [ ]:
plt.bar(grid_dists)

In [214]:
qmc_dists

tensor([15575.2344,  1041.2139,   471.6435,   106.5965,    59.1255,    45.8519,
           41.1897,    39.6904])

In [208]:
front_g_m = torch.stack([m[0].float().mean() for m in grid_masks], dim=0)
front_g_s = torch.stack([m[0].float().std() for m in grid_masks], dim=0)
back_g_m = torch.stack([m[1].float().mean() for m in grid_masks], dim=0)
back_g_s = torch.stack([m[1].float().std() for m in grid_masks], dim=0)

In [209]:
front_g_m, front_g_s

(tensor([  1.0000,  10.5625,   8.6289,  15.0156,  46.4102,  60.5479, 160.5764,
         196.0000]),
 tensor([ 0.0000,  3.0957,  1.0093,  4.7626,  6.1273, 22.8689, 20.5887, 87.4629]))

In [210]:
back_g_m, back_g_s

(tensor([  1.0000,   2.2500,   7.5625,   7.5625,  39.0625,  31.6406, 136.5977,
         142.5039]),
 tensor([ 0.0000,  1.1255,  1.7078,  1.7078,  9.7238,  6.8699, 33.1658, 35.0628]))

In [185]:
front_q_m = torch.stack([m[0].float().mean() for m in qmc_masks], dim=0)
front_q_s = torch.stack([m[0].float().std() for m in qmc_masks], dim=0)
back_q_m = torch.stack([m[1].float().mean() for m in qmc_masks], dim=0)
back_q_s = torch.stack([m[1].float().std() for m in qmc_masks], dim=0)

In [190]:
front_q_m, front_q_s

(tensor([  1.8906,   4.7576,   9.3281,  18.5356,  37.3281,  74.0309, 146.2527,
         292.7811]),
 tensor([ 0.6695,  1.0991,  1.6431,  2.6684,  4.9519,  9.4219, 18.3863, 36.7924]))

In [191]:
back_q_m, back_q_s

(tensor([  3.2500,   4.6970,   8.4375,  15.3492,  31.3125,  63.8085, 126.1699,
         250.4597]),
 tensor([ 1.3416,  1.3803,  2.1958,  3.7912,  7.4199, 15.3288, 30.1571, 59.7511]))

### RQMC variance

In [ ]:
base_logits, logits, masks = torch.load(osp.expanduser('~/code/diffcoord/temp/output_variance_rqmc.pt'))

In [160]:
logit_dists = (torch.cat(logits, dim=0) - base_logits.cpu()).norm(dim=-1)

In [ ]:
logit_dists

In [ ]:
fronts.mean(1), fronts.std(1), fronts.amin(1), fronts.amax(1)

In [ ]:
backs.mean(1), backs.std(1), backs.amin(1), backs.amax(1)

In [157]:
fronts = torch.stack([m[0] for m in masks], dim=0).float()
backs = torch.stack([m[1] for m in masks], dim=0).float()

### grid-QMC interpolation

In [ ]:
base_logits, grid_logits, grid_mask, qmc_logits, qmc_mask, intermediate_logits, intermediate_masks = torch.load(osp.expanduser('~/code/diffcoord/temp/analyze_logit_mismatch.pt'))

x = np.arange(0,1.01,.05)
fronts = torch.stack([grid_mask[0], *[m[0] for m in intermediate_masks[::-1]], qmc_mask[0]], dim=0).float()
backs = torch.stack([grid_mask[1], *[m[1] for m in intermediate_masks[::-1]], qmc_mask[1]], dim=0).float()
f_m, f_s = fronts.mean(1), fronts.std(1)
b_m, b_s = backs.mean(1), backs.std(1)

fig,ax = plt.subplots()
width = .007
ax.errorbar(x-width/2, f_m, yerr=f_s, label='front')
ax.errorbar(x+width/2, b_m, yerr=b_s, label='back')

In [ ]:
y = (torch.cat([grid_logits, *intermediate_logits[::-1], qmc_logits], dim=0) - base_logits.cpu()).norm(dim=1)
plt.plot(x,y)

# slurm

In [45]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334
inet_len = 50000
in_tr_len = 50000

In [ ]:
configs/convnext/upernet_convnext_tiny
configs/_base_/models/upernet_convnext.py

In [ ]:
for ix in range(0,places_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/places/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"place{ix//800}", "fit_places", ix)

In [ ]:
for ix in range(0,in_tr_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_train/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_in_tr", ix)

In [ ]:
for ix in range(0,60000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/fmnist/train_{ix+1}.pt"):
        print("sh fit_inr.sh", f"fmn{ix//5000}", "fit_fmnist", ix)

In [ ]:
for ix in range(0,12000,100):
    c,i=ix//1000, ix%1000
    if i >= 800:
        i-=800
        s='val'
    else:
        s='train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{c}/{s}_{i+3}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

In [35]:
for ix in range(0,2400,50):
    c,i = ix%12, ix//12
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{c}/test_{i+1}.pt"):
        print("sh fit_inr.sh", f"rff{ix//50}", "fit_in_rff", ix)

In [ ]:
#23400 * 12 = 280800 total images
#15000/12 + 800 = 2050 current images per class
for ix in range(36000,48000,100):
    cls, start_ix = ix % 12, ix // 12 + 800
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{cls}/train_{start_ix+4}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

In [ ]:
for ix in range(0,8189,100):
    i = ix
    if i < 1020:
        s='train'
    elif i < 2040:
        s='val'
        i -= 1020
    else:
        s='test'
        i -= 2040
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/flowers/{s}_{i+50}.pt"):
        print("sh fit_inr.sh", f"flow{ix//100}", "fit_flower", ix)

In [ ]:
for ix in range(3475,4475,100):
    i = ix
    if i >= 2975:
        i -= 2975
        s = 'val'
    else:
        s = 'train'
        i -= 500
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/{s}_{i+1}.pt"):
        print("sh fit_inr.sh", f"city{ix//100}", "fit_city", ix)

In [46]:
for ix in range(0,inet_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_val/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_inet", ix)

In [37]:
x[0,0] = 1

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [ ]:
for ix in range(0,10000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_test/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cit{ix//5000}", "fit_ciftest", ix)
for ix in range(0,50000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_train/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cif{ix//5000}", "fit_cifar", ix)

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/1/loss_*")

In [ ]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/0/loss_train_10*")
for p in paths[10:100]:
    print(open(p,'r').read())